In [ ]:
import textstat
import pandas as pd
import spacy
import subprocess
subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])

In [42]:
# Load English SpaCy model
nlp = spacy.load("en_core_web_sm") #In terminal: python -m spacy download en_core_web_sm

# Download concreteness lexicon
# Direct URL to the CSV file (latest version, Conc.M = concreteness mean)
url = "hf://datasets/StephanAkkerman/MRC-psycholinguistic-database/mrc_psycholinguistic_database.csv"
# Load into pandas
lexicon = pd.read_csv(url)[['Word','Concreteness', 'Imageability']]
lexicon["Word"] = lexicon["Word"].str.lower()
lexicon.head()

,Word,Concreteness,Imageability
0,&arry,0,0
1,&cello,0,0
2,&d,0,0
3,&em,0,0
4,&flu,0,0


In [ ]:
# Define the function

def analyze_text(text):
    doc = nlp(text)
    words = [token.text.lower() for token in doc if token.is_alpha]

    # Readability (Flesch Reading Ease)
    readability = textstat.flesch_reading_ease(text)

    # Imageability & Concreteness (averages)
    imageability_scores = [
        lexicon.loc[lexicon['Word'] == word, 'Imageability'].values[0]
        for word in words if word in lexicon['Word'].values and 'Imageability' in lexicon.columns
    ]
    concreteness_scores = [
        lexicon.loc[lexicon['Word'] == word, 'Concreteness'].values[0]
        for word in words if word in lexicon['Word'].values and 'Concreteness' in lexicon.columns
    ]

    avg_imageability = sum(imageability_scores) / len(imageability_scores) if imageability_scores else None
    avg_concreteness = sum(concreteness_scores) / len(concreteness_scores) if concreteness_scores else None

    # Syntactic simplicity (average number of words per sentence)
    avg_sentence_length = sum(len(sent) for sent in doc.sents) / len(list(doc.sents))

    return {
        "Readability": round(readability, 2),
        "Imageability": round(avg_imageability, 2) if avg_imageability else "N/A",
        "Concreteness": round(avg_concreteness, 2) if avg_concreteness else "N/A",
        "Syntactic simplicity (avg sentence length)": round(avg_sentence_length, 2)
    }


In [30]:
# Exemple d'utilisation
poems=pd.read_excel("/home/onyxia/work/Phoetry/generated_poems/poem_classic_.xlsx")

poems.head()

,theme,text
0,moon,Then we see the moon shining on her face. It's...
1,leaf,I only I could have the leaf. But it's not lik...
2,flower,For I am the flower of life. And that is what ...
3,tree,Then we see the tree fall down to its own litt...
4,sun,I only I could have the sun. When it was dark ...


In [31]:
poems["Readability"]=poems["text"].apply(lambda x: analyze_text(x)["Readability"])
poems["Imageability"]=poems["text"].apply(lambda x: analyze_text(x)["Imageability"])
poems["Concreteness"]=poems["text"].apply(lambda x: analyze_text(x)["Concreteness"])
poems["Syntactic simplicity"]=poems["text"].apply(lambda x: analyze_text(x)["Syntactic simplicity (avg sentence length)"])
poems


,theme,text,Readability,Imageability,Concreteness,Syntactic simplicity
0,moon,Then we see the moon shining on her face. It's...,101.29,329.57,298.64,14.75
1,leaf,I only I could have the leaf. But it's not lik...,99.57,305.75,274.91,21.00
2,flower,For I am the flower of life. And that is what ...,85.73,279.95,251.09,20.33
3,tree,Then we see the tree fall down to its own litt...,89.08,293.44,261.56,18.00
4,sun,I only I could have the sun. When it was dark ...,99.57,288.22,267.02,16.50
5,sunset,For I am the sunset; for it is my light that s...,85.02,300.74,271.41,21.75
6,waterfall,Then we see the waterfall that is at first. It...,69.75,263.41,232.03,28.33
7,butterfly,I only I could have the butterfly. When it was...,90.29,304.07,265.86,17.40
8,bird,Then we see the bird come out of its shell. Th...,90.29,286.56,249.60,17.40
9,dog,For I am the dog who is to be eaten by wolves....,84.00,294.23,245.16,23.00


In [ ]:
poems.describe()

,Readability,Imageability,Concreteness,Syntactic simplicity
count,13.000000,13.000000,13.000000,13.000000
mean,87.191538,293.650769,261.428462,20.073846
std,10.792160,19.875319,18.767763,4.303194
min,64.340000,261.060000,232.030000,14.670000
25%,84.000000,286.560000,249.600000,17.400000
50%,89.080000,293.440000,261.560000,20.330000
75%,90.900000,304.070000,271.410000,21.750000
max,101.290000,329.570000,298.640000,28.330000


In [34]:
poems.to_excel("Scores_poems.xlsx")